In [4]:
from rlr_validate_custom import rlr_validate_custom
from sklearn import model_selection
from ann_validate import ann_validate
import torch
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from scipy.stats import zscore

url = "https://hastie.su.domains/ElemStatLearn/datasets/SAheart.data"

# Load the SAheart dataset
df = pd.read_csv(url, index_col='row.names')

# Convert binary text data to numbered categories
df['famhist'] = pd.Categorical(df['famhist']).codes

# Extract the target attribute, and remove it from the training data
y = np.asarray(np.asmatrix(df["typea"].values).T).squeeze()
df = df.drop(columns=["typea"])

# Attribute names
attributeNames = list(map(lambda x: x.capitalize(), df.columns.tolist()))

# Convert the training data to a numpy array
X = df.to_numpy()
N, M = X.shape

X = zscore(X, ddof=1)  # Mean = 0, Std = 1
y = zscore(y, ddof=1)  # Mean = 0, Std = 1

# ---
# End of data loading
# ---


# Add offset attribute
X = np.concatenate((np.ones((X.shape[0], 1)), X), 1)
attributeNames = ["Offset"] + attributeNames
M = M + 1

## Crossvalidation
# Create crossvalidation partition for evaluation
K = 10
CV = model_selection.KFold(K, shuffle=True)
# CV = model_selection.KFold(K, shuffle=False)

lambda_count = 10

# Values of lambda
lambdas = np.logspace(0, 4, lambda_count)
n_hidden_units_range = range(1, 30)
max_iter = 10000


# Linear regression
lr_error_train = np.empty((K, 1))
lr_error_test = np.empty((K, 1))
best_lambda = np.empty((K, 1))

# ANN
ann_error_train = np.empty((K, 1))
ann_error_test = np.empty((K, 1))
best_hidden_units = np.empty((K, 1))

# Baseline
baseline_error_test = np.empty((K, 1))


w_rlr = np.empty((M, K))
mu = np.empty((K, M - 1))
sigma = np.empty((K, M - 1))
w_noreg = np.empty((M, K))

k = 0
for train_index, test_index in CV.split(X, y):
    # extract training and test set for current CV fold
    X_train = X[train_index]
    y_train = y[train_index]
    X_test = X[test_index]
    y_test = y[test_index]

    innerCV = model_selection.KFold(K, shuffle=True)


    # ----
    # Linear regression
    # ----

    (   opt_val_err,
        opt_lambda,
        mean_w_vs_lambda,
        train_err_vs_lambda,
        test_err_vs_lambda,
    ) = rlr_validate_custom(X_train, y_train, lambdas, K, innerCV)

    # Standardize outer fold based on training set, and save the mean and standard
    # deviations since they're part of the model (they would be needed for
    # making new predictions) - for brevity we won't always store these in the scripts
    mu[k, :] = np.mean(X_train[:, 1:], 0)
    sigma[k, :] = np.std(X_train[:, 1:], 0)

    X_train[:, 1:] = (X_train[:, 1:] - mu[k, :]) / sigma[k, :]
    X_test[:, 1:] = (X_test[:, 1:] - mu[k, :]) / sigma[k, :]

    Xty = X_train.T @ y_train
    XtX = X_train.T @ X_train

    # Estimate weights for the optimal value of lambda, on entire training set
    lambdaI = opt_lambda * np.eye(M)
    lambdaI[0, 0] = 0  # Do no regularize the bias term
    w_rlr[:, k] = np.linalg.solve(XtX + lambdaI, Xty).squeeze()

    # Compute mean squared error with regularization with optimal lambda
    lr_error_train[k] = (
        np.square(y_train - X_train @ w_rlr[:, k]).sum(axis=0) / y_train.shape[0]
    )
    lr_error_test[k] = (
        np.square(y_test - X_test @ w_rlr[:, k]).sum(axis=0) / y_test.shape[0]
    )
    best_lambda[k] = opt_lambda

    # ----
    # ANN
    # ----

    loss_fn = torch.nn.MSELoss()
    models = np.empty((len(n_hidden_units_range),), dtype=object)

    # Generate out models with different number of hidden units
    for i, n_hidden_units in enumerate(n_hidden_units_range):
        model = torch.nn.Sequential(
            torch.nn.Linear(M, n_hidden_units),  # Input layer
            torch.nn.Tanh(),  # Hidden activation (ReLU is also good)
            torch.nn.Linear(n_hidden_units, 1)   # Output layer (NO SIGMOID for regression)
        )
        models[i] = model

    (
        opt_val_err,
        opt_model_index,
        opt_model,
        opt_network,
        train_err_vs_lambda,
        test_err_vs_lambda
        #This does a folding cross-validation
    ) = ann_validate(torch.Tensor(X_train), torch.Tensor(y_train), models, loss_fn, 1, max_iter, K, innerCV)

    #ann_error_train[k] = np.square(y_train-opt_network(torch.Tensor(X_train)).detach().numpy()).sum()/y_train.shape[0]
    #ann_error_test[k] = np.square(y_train-opt_network(torch.Tensor(X_train)).detach().numpy()).sum()/y_train.shape[0]

    ann_error_train[k] = train_err_vs_lambda[opt_model_index]
    ann_error_test[k] = test_err_vs_lambda[opt_model_index]
    best_hidden_units[k] = n_hidden_units_range[opt_model_index]

    # ----
    # Baseline
    # ----

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Compute the mean of y_train as the baseline prediction
    y_baseline = np.mean(y_train)

    # Predict the same mean value for all test samples
    y_pred_baseline = np.full_like(y_test, y_baseline)

    # Compute and store the MSE
    mse = mean_squared_error(y_test, y_pred_baseline)
    baseline_error_test[k] = mse


    k += 1


[Sequential(
   (0): Linear(in_features=10, out_features=1, bias=True)
   (1): Tanh()
   (2): Linear(in_features=1, out_features=1, bias=True)
 )
 Sequential(
   (0): Linear(in_features=10, out_features=2, bias=True)
   (1): Tanh()
   (2): Linear(in_features=2, out_features=1, bias=True)
 )
 Sequential(
   (0): Linear(in_features=10, out_features=3, bias=True)
   (1): Tanh()
   (2): Linear(in_features=3, out_features=1, bias=True)
 )
 Sequential(
   (0): Linear(in_features=10, out_features=4, bias=True)
   (1): Tanh()
   (2): Linear(in_features=4, out_features=1, bias=True)
 )
 Sequential(
   (0): Linear(in_features=10, out_features=5, bias=True)
   (1): Tanh()
   (2): Linear(in_features=5, out_features=1, bias=True)
 )
 Sequential(
   (0): Linear(in_features=10, out_features=6, bias=True)
   (1): Tanh()
   (2): Linear(in_features=6, out_features=1, bias=True)
 )
 Sequential(
   (0): Linear(in_features=10, out_features=7, bias=True)
   (1): Tanh()
   (2): Linear(in_features=7, out_feat

/Users/dhanna/Documents/Egyetem/DTU/1_Semester/IntroductionToMachineLearning/Report/Report_1/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([373])) that is different to the input size (torch.Size([373, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/dhanna/Documents/Egyetem/DTU/1_Semester/IntroductionToMachineLearning/Report/Report_1/.venv/lib/python3.13/site-packages/dtuimldmtools/models/nn_trainer.py:141: RuntimeWarning: overflow encountered in cast
  if loss_value < best_final_loss:


		1000	0.9131835	2.1147443e-05
		Final loss:
		1699	0.9089914	9.835834e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.910135	3.2089947e-06
		Final loss:
		1493	0.909272	9.832797e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.9093191	1.7698105e-06
		Final loss:
		1200	0.90906584	9.835028e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.9170718	1.1633884e-05
		2000	0.9116975	3.0727404e-06
		3000	0.9098432	1.2447055e-06
		Final loss:
		3182	0.90964645	9.82875e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.9097102	2.0966509e-06
		Final loss:
		1257	0.9093605	9.831841e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.9104422	2.8151073e-06
		Final loss:
		1542	0.90955156	9.829776e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.9096844	3.9313245e-06
		Final loss:
		1353	0.9089558	9.836218e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.9102835	3.6668223e-06
		Final loss:
		1478	0.9093787	9.831645e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.9

/Users/dhanna/Documents/Egyetem/DTU/1_Semester/IntroductionToMachineLearning/Report/Report_1/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([374])) that is different to the input size (torch.Size([374, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


		Final loss:
		1627	0.9751644	9.779617e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.976598	6.5304816e-06
		Final loss:
		1468	0.9752407	9.778852e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.97591037	2.7484098e-06
		Final loss:
		1279	0.9754218	9.777036e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.9765643	2.2582915e-06
		Final loss:
		1532	0.97572875	9.77396e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.97563815	2.504806e-06
		Final loss:
		1225	0.9752576	9.778682e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.9790432	1.3210895e-05
		Final loss:
		1838	0.9755469	9.775782e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.98007464	8.453413e-06
		2000	0.97596455	1.7100285e-06
		Final loss:
		2259	0.97561693	9.775081e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.9775712	6.5239806e-06
		Final loss:
		1661	0.9756259	9.774991e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.97614026	2.62564e-06
		Final loss:
		1311	0.97559214	9.775329

...

		Final loss:
		979	1.0251052	9.303176e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	1.0340391	2.0405043e-05
		2000	1.0260055	2.4399371e-06
		Final loss:
		2304	1.0254892	9.2996925e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	1.0276968	4.9878267e-06
		Final loss:
		1784	1.0256579	9.298163e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	1.0260861	1.5103201e-06
		Final loss:
		1206	1.0258248	9.29665e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	1.0399896	3.6449568e-05
		2000	1.0268388	2.55405e-06
		Final loss:
		2497	1.0259738	9.2953e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	1.0263721	2.32292e-06
		Final loss:
		1365	1.0257666	9.2971777e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	1.0264775	4.0646855e-06
		Final loss:
		1446	1.0254469	9.300076e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	1.0263162	3.0199585e-06
		Final loss:
		1380	1.0256169	9.2985346e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	1.0288088	7.4157e-06
		Final loss:
		18

/Users/dhanna/Documents/Egyetem/DTU/1_Semester/IntroductionToMachineLearning/Report/Report_1/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([374])) that is different to the input size (torch.Size([374, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/dhanna/Documents/Egyetem/DTU/1_Semester/IntroductionToMachineLearning/Report/Report_1/.venv/lib/python3.13/site-packages/dtuimldmtools/models/nn_trainer.py:141: RuntimeWarning: overflow encountered in cast
  if loss_value < best_final_loss:


		1000	0.95911276	1.0378209e-05
		2000	0.95547307	1.185263e-06
		Final loss:
		2048	0.95542014	9.981717e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.956053	2.244397e-06
		Final loss:
		1301	0.9555906	9.979937e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.9555477	1.6218115e-06
		Final loss:
		1085	0.9554379	9.981532e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.95965356	1.1490326e-05
		2000	0.9557606	1.1849064e-06
		Final loss:
		2049	0.955705	9.978742e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.95780724	7.2808934e-06
		Final loss:
		1666	0.9556766	9.979038e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.95610297	3.0547114e-06
		Final loss:
		1340	0.9554743	9.981151e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.95695674	4.173119e-06
		Final loss:
		1530	0.95582396	9.9775e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.9565507	2.3055409e-06
		Final loss:
		1418	0.95587635	9.976953e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0

/Users/dhanna/Documents/Egyetem/DTU/1_Semester/IntroductionToMachineLearning/Report/Report_1/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([375])) that is different to the input size (torch.Size([375, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


		1000	0.94045323	8.492665e-06
		Final loss:
		1503	0.9387952	9.5235765e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.9510788	2.5380936e-05
		2000	0.9410011	4.117196e-06
		Final loss:
		2881	0.9390471	9.521022e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.9402811	4.500687e-06
		Final loss:
		1485	0.9392158	9.519312e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.9397102	2.28343e-06
		Final loss:
		1372	0.9391749	9.5197265e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.941199	6.269474e-06
		Final loss:
		1747	0.939238	9.519087e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.93993735	4.2486813e-06
		Final loss:
		1374	0.93913424	9.5201386e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.9399621	3.233989e-06
		Final loss:
		1406	0.9392527	9.518938e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.942974	6.889757e-06
		2000	0.93972343	1.5222663e-06
		Final loss:
		2203	0.9394773	9.516662e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.94026

In [5]:
results = pd.DataFrame({
    "h" : best_hidden_units.flatten(),
    "ann_error_test": ann_error_test.flatten(),

    "lambda": best_lambda.flatten(),
    "lr_error_test": lr_error_test.flatten(),

    "baseline_error_test": baseline_error_test.flatten(),
})
results

,h,ann_error_test,lambda,lr_error_test,baseline_error_test
0,17.0,0.971768,21.544347,1.487239,1.166771
1,15.0,1.003131,166.810054,0.906316,0.940448
2,24.0,1.003392,166.810054,0.864727,0.907428
3,9.0,0.982390,59.948425,1.190462,1.134726
4,17.0,0.994919,59.948425,0.944921,0.997473
5,22.0,1.034002,59.948425,0.610766,0.605468
6,10.0,1.004867,166.810054,0.818156,0.913659
7,28.0,1.005213,166.810054,0.847721,0.890293
8,10.0,1.015483,166.810054,0.821725,0.865845
9,24.0,0.933886,166.810054,1.499995,1.573426


In [10]:
latex_table = r"""\begin{tabular}{c|cc|cc|c}
\toprule
\textbf{Outer fold} & \multicolumn{2}{c|}{\textbf{ANN}} & \multicolumn{2}{c|}{\textbf{Linear regression}} & \textbf{baseline} \\
$i$ & $h_i^*$ & $E_i^{\text{test}}$ & $\lambda_i^*$ & $E_i^{\text{test}}$ & $E_i^{\text{test}}$ \\
\midrule
"""

for i, row in results.iterrows():
    latex_table += f"{i+1} & {row['h']} & {row['ann_error_test']:.1f} & {row['lambda']:.2f} & {row['lr_error_test']:.1f} & {row['baseline_error_test']:.1f} \\\\\n"

latex_table += r"\bottomrule" + "\n" + r"\end{tabular}"
print(latex_table)


\begin{tabular}{c|cc|cc|c}
\toprule
\textbf{Outer fold} & \multicolumn{2}{c|}{\textbf{ANN}} & \multicolumn{2}{c|}{\textbf{Linear regression}} & \textbf{baseline} \\
$i$ & $h_i^*$ & $E_i^{\text{test}}$ & $\lambda_i^*$ & $E_i^{\text{test}}$ & $E_i^{\text{test}}$ \\
\midrule
1 & 17.0 & 1.0 & 21.54 & 1.5 & 1.2 \\
2 & 15.0 & 1.0 & 166.81 & 0.9 & 0.9 \\
3 & 24.0 & 1.0 & 166.81 & 0.9 & 0.9 \\
4 & 9.0 & 1.0 & 59.95 & 1.2 & 1.1 \\
5 & 17.0 & 1.0 & 59.95 & 0.9 & 1.0 \\
6 & 22.0 & 1.0 & 59.95 & 0.6 & 0.6 \\
7 & 10.0 & 1.0 & 166.81 & 0.8 & 0.9 \\
8 & 28.0 & 1.0 & 166.81 & 0.8 & 0.9 \\
9 & 10.0 & 1.0 & 166.81 & 0.8 & 0.9 \\
10 & 24.0 & 0.9 & 166.81 & 1.5 & 1.6 \\
\bottomrule
\end{tabular}
